In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("pkmn.csv")
df.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80
3,4,Charmander,Fire,NaN,39,52,43,60,50,65
4,5,Charmeleon,Fire,NaN,58,64,58,80,65,80


In [ ]:
df.describe()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
count,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000
mean,247.000000,67.730223,73.496957,70.109533,67.981744,69.158215,65.440162
std,142.461106,27.580375,29.168464,30.703012,28.515038,27.884112,27.223685
min,1.000000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000
25%,124.000000,50.000000,50.000000,50.000000,45.000000,50.000000,45.000000
50%,247.000000,65.000000,72.000000,65.000000,65.000000,65.000000,65.000000
75%,370.000000,80.000000,90.000000,85.000000,90.000000,85.000000,85.000000
max,493.000000,255.000000,165.000000,230.000000,154.000000,230.000000,160.000000


In [37]:
# Preprocesamiento del DataFrame
def preprocess_dataframe(df):
    df_processed = df.copy()

    # Codificación One-Hot para 'Type 1' y 'Type 2'
    # Esta es la parte crucial donde Type 2 se convierte en características para la similitud
    df_processed = pd.get_dummies(df_processed, columns=['Type 1', 'Type 2'], prefix=['Type1', 'Type2'], dummy_na=False)

    # Escalado de características numéricas
    numeric_cols = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
    scaler = StandardScaler()
    df_processed[numeric_cols] = scaler.fit_transform(df_processed[numeric_cols])
    
    return df_processed, scaler, numeric_cols

df_processed, scaler, numeric_cols = preprocess_dataframe(df)

def recommend_pokemons(df, df_processed, scaler, numeric_cols, user_type1, user_type2=None, user_hp=None, user_attack=None, user_defense=None, user_sp_atk=None, user_sp_def=None, user_speed=None, top_n=3):
    """
    Recomienda los Pokémon más similares basados en los tipos y estadísticas del usuario,
    dando prioridad a Type 1 y luego considerando Type 2 y estadísticas.
    Devuelve los resultados por su número de Pokedex (#).

    Args:
        df (pd.DataFrame): El DataFrame original de Pokémon.
        df_processed (pd.DataFrame): El DataFrame preprocesado con codificación dummy y escalado.
        scaler (StandardScaler): El escalador utilizado para las características numéricas.
        numeric_cols (list): Lista de nombres de columnas numéricas.
        user_type1 (str): El tipo principal del usuario (requerido).
        user_type2 (str, optional): El segundo tipo del usuario. Por defecto es None.
        user_hp (int, optional): HP deseado por el usuario.
        user_attack (int, optional): Ataque deseado por el usuario.
        user_defense (int, optional): Defensa deseada por el usuario.
        user_sp_atk (int, optional): Ataque Especial deseado por el usuario.
        user_sp_def (int, optional): Defensa Especial deseada por el usuario.
        user_speed (int, optional): Velocidad deseada por el usuario.
        top_n (int): Número de Pokémon a recomendar. Por defecto es 3.

    Returns:
        pd.DataFrame: Una tabla Markdown con los Pokémon recomendados por su número de Pokedex.
    """

    # Crear un DataFrame para las preferencias del usuario
    user_data = {
        'HP': [user_hp if user_hp is not None else df['HP'].mean()],
        'Attack': [user_attack if user_attack is not None else df['Attack'].mean()],
        'Defense': [user_defense if user_defense is not None else df['Defense'].mean()],
        'Sp. Atk': [user_sp_atk if user_sp_atk is not None else df['Sp. Atk'].mean()],
        'Sp. Def': [user_sp_def if user_sp_def is not None else df['Sp. Def'].mean()],
        'Speed': [user_speed if user_speed is not None else df['Speed'].mean()]
    }
    user_df = pd.DataFrame(user_data)

    # Escalar las características numéricas del usuario
    user_df[numeric_cols] = scaler.transform(user_df[numeric_cols])

    # Preparar el vector de características del usuario
    # Excluir la columna '#' del dataframe procesado para la comparación de características
    pokemon_features_df = df_processed.drop(columns=['Name', '#'], errors='ignore') 
    user_features = pd.DataFrame(0, index=[0], columns=pokemon_features_df.columns)

    # Asignar los tipos del usuario
    type1_col = f'Type1_{user_type1}'
    if type1_col in user_features.columns:
        user_features[type1_col] = 1
    else:
        print(f"Advertencia: El tipo '{user_type1}' no se encontró en los datos de entrenamiento para Type 1.")

    if user_type2:
        type2_col = f'Type2_{user_type2}'
        if type2_col in user_features.columns:
            user_features[type2_col] = 1
        else:
            print(f"Advertencia: El tipo '{user_type2}' no se encontró en los datos de entrenamiento para Type 2.")

    # Asignar las estadísticas numéricas del usuario escaladas
    for col in numeric_cols:
        user_features[col] = user_df[col].iloc[0]

    # Calcular la similitud del coseno
    similarities = cosine_similarity(user_features, pokemon_features_df)

    # Obtener los índices de los Pokémon más similares (índices internos de Pandas)
    similar_pokemon_internal_indices = similarities.argsort()[0][::-1]

    # Filtrar por Type 1 primero, y luego considerar Type 2
    final_recommended_internal_indices = []
    
    # Primero, intenta encontrar Pokémon que coincidan con user_type1
    temp_type1_matches = []
    for internal_idx in similar_pokemon_internal_indices:
        if df.loc[internal_idx, 'Type 1'] == user_type1:
            temp_type1_matches.append(internal_idx)
    
    # Ahora, de esos matches de Type 1, prioriza los que también coinciden con user_type2 (si existe)
    if user_type2:
        temp_type1_type2_matches = []
        temp_type1_only_matches = []
        for internal_idx in temp_type1_matches:
            if df.loc[internal_idx, 'Type 2'] == user_type2:
                temp_type1_type2_matches.append(internal_idx)
            else:
                temp_type1_only_matches.append(internal_idx)
        
        # Agrega primero los que coinciden con ambos tipos, hasta top_n
        final_recommended_internal_indices.extend(temp_type1_type2_matches[:top_n])
        
        # Si aún faltan, agrega los que solo coinciden con Type 1
        if len(final_recommended_internal_indices) < top_n:
            remaining_needed = top_n - len(final_recommended_internal_indices)
            final_recommended_internal_indices.extend(temp_type1_only_matches[:remaining_needed])
            
    else: # Si user_type2 no se especifica, solo toma los mejores por Type 1
        final_recommended_internal_indices.extend(temp_type1_matches[:top_n])


    # Si aún no hemos alcanzado top_n después de priorizar Type 1 (y Type 2),
    # agregamos los Pokémon más similares en general (basado en coseno, que incluye ambos tipos y stats)
    if len(final_recommended_internal_indices) < top_n:
        remaining_needed = top_n - len(final_recommended_internal_indices)
        count_added = 0
        for internal_idx in similar_pokemon_internal_indices:
            if internal_idx not in final_recommended_internal_indices:
                final_recommended_internal_indices.append(internal_idx)
                count_added += 1
            if count_added >= remaining_needed:
                break
    
    # Asegurarse de que no haya duplicados y limitar a top_n en caso de que la lógica anterior haya agregado de más
    final_recommended_internal_indices = list(dict.fromkeys(final_recommended_internal_indices))[:top_n]


    # Obtener el DataFrame de los Pokémon recomendados usando los índices internos finales
    recommended_pokemons = df.loc[final_recommended_internal_indices].copy()
    
    # Calcular la similitud para la tabla resumen
    recommended_pokemons['Similarity'] = [similarities[0][idx] for idx in final_recommended_internal_indices]


    # Formatear la tabla en Markdown
    markdown_table = "### Pokémon Recomendados (por número de Pokedex)\n\n"
    markdown_table += "| # | Name | Type 1 | Type 2 | HP | Attack | Defense | Sp. Atk | Sp. Def | Speed | Similitud |\n"
    markdown_table += "|---|---|---|---|---|---|---|---|---|---|---|\n"
    for index, row in recommended_pokemons.iterrows():
        type2_val = row['Type 2'] if pd.notna(row['Type 2']) else 'N/A'
        markdown_table += (
            f"| {int(row['#'])} | {row['Name']} | {row['Type 1']} | {type2_val} | {row['HP']} | {row['Attack']} | {row['Defense']} | "
            f"{row['Sp. Atk']} | {row['Sp. Def']} | {row['Speed']} | {row['Similarity']:.4f} |\n"
        )
    return markdown_table

In [43]:
# --- Ejemplo de uso ---
# Supongamos que el usuario quiere Pokémon tipo 'Water' y 'Psychic' con stats altos
# # Recomendación de Pokémon
print(recommend_pokemons(df, df_processed, scaler, numeric_cols,
                         user_type1='Bug', 
                         user_type2='Water', # Ahora Type 2 también es un factor en el filtrado
                         user_hp=40, 
                         user_attack=100, 
                         user_defense=80, 
                         user_sp_atk=40, 
                         user_sp_def=70, 
                         user_speed=110))

### Pokémon Recomendados (por número de Pokedex)

| # | Name | Type 1 | Type 2 | HP | Attack | Defense | Sp. Atk | Sp. Def | Speed | Similitud |
|---|---|---|---|---|---|---|---|---|---|---|
| 283 | Surskit | Bug | Water | 40 | 30 | 32 | 50 | 52 | 65 | 0.2414 |
| 123 | Scyther | Bug | Flying | 70 | 110 | 80 | 55 | 80 | 105 | 0.7318 |
| 291 | Ninjask | Bug | Flying | 61 | 90 | 45 | 50 | 50 | 160 | 0.7067 |

